In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# using the ensamble method to predict 
from sklearn.ensemble import VotingClassifier

ensemble_model = VotingClassifier(estim ators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')
ensemble_model.fit(X_train, y_train)

# ocena jakości modelu
score = ensemble_model.score(X_test, y_test)
print("Accuracy: %.2f%%" % (score * 100))